In [5]:
import copy
import os
import zipfile


class Person(object):
    def __init__(self, name, number):
        self.name = name
        self.num = number


class ReadData(object):
    @classmethod
    def create_from_txt(cls, path, mode):
        obj = cls()
        with open(path, mode, encoding='UTF-8') as f:
            for line in f.readlines():
                line = line.rstrip()
                persondata = line.split()
                obj.append(Person(name=persondata[0], number=int(persondata[1])))
        return obj

    @classmethod
    def create_from_csv(cls, path, mode):
        obj = cls()
        with open(path, mode, encoding='gbk') as f:
            for line in f.readlines():
                line = line.rstrip()
                persondata = line.split(',')
                obj.append(Person(name=persondata[0], number=int(persondata[1])))
        return obj

    @classmethod
    def read_zip(cls, path, mode):
        z_file = zipfile.ZipFile(path, mode)
        flist = z_file.namelist()
        extract_file = z_file.extract(flist[0])
        return extract_file

    # 静态方法 读取文件名字和类型
    @staticmethod
    def read_filename(path):
        file = os.path.splitext(path)
        filename, filetype = file
        return filename, filetype

    # 从文件中创建一个对象
    @classmethod
    def create_from_file(cls, path, mode):
        filename, filetype = cls.read_filename(path)
        if filetype == '.zip':
            filepath = cls.read_zip(path, mode)
            # 嵌套中必须有return 否则为空
            filez = cls.create_from_file(filepath, mode)
            return filez
        if filetype == '.txt':
            return cls.create_from_txt(path, mode)
        if filetype == '.csv':
            return cls.create_from_csv(path, mode)

    # 子类的append会覆盖父类的append 此处写上防止报错        
    def append(self, obj):
        return

# 继承父类的create_from_file()方法
class Ring(ReadData):
    MAX_SIZE = 1000

    def __init__(self):
        self.step = 1
        self.start = 0
        self.__ring = []
        self.__temp = []
        self.__current_id = 0

    def is_empty(self):
        return len(self.__temp) == 0

    def pop(self, index):
        self.__ring.pop(index)

    def quer_list(self):
        return self.__ring

    def reset(self, step, location):
        self.step = step
        self.start = location-1
        self.__current_id = self.start
        self.__temp = copy.deepcopy(self.__ring)
        return

    def next(self):
        if self.is_empty() is True:
            raise Exception('环为空')
        size = len(self.__temp)
        id = (self.__current_id+(self.step-1)) % size
        outelem = self.__temp.pop(id)
        self.__current_id += (self.step - 1)
        return outelem

    def popelem(self):
        step = self.step
        temp = self.__temp
        location = self.start % len(temp)
        if self.is_empty() is True:
            raise Exception('环为空')

        for each in range(0, len(temp)):
            location = (location + (step-1)) % len(temp)
            outelem = temp.pop(location)
            yield outelem

    def append(self, obj):
        if len(self.__ring) > Ring.MAX_SIZE:
            raise Exception("out of range")
        self.__ring.append(obj)


try:
    steps = int(input("数几个数"))
    if steps <= 0:
        raise Exception('数的数必须大于1')
    loc = int(input("开始位置"))
    if loc <= 0:
        raise Exception('位置必须大于1')

    jospeh = Ring.create_from_file('peopledata.zip', 'r')
    jospeh.reset(steps, loc)

except ValueError:
    print('输入错误，输入的不是数字')
else:
    popseq = jospeh.popelem()
    for popperson in popseq:
        print(popperson.name, popperson.num)


金迎夜 5
凤千谷 8
冯巧 1
吕天 4
杨岚 9
袁春绿 3
陶晓 10
尤文白 7
昌瑶灵 2
潘海曼 6


In [2]:
class Node(object):
    def __init__(self, item, name):
        self.elem = item
        self.next = None
        self.name = name


class SingleCycleLinkList(object):
    """单向循环链表"""

    def __init__(self):
        self.head = None

    def is_empty(self):
        # 链表是否为空
        return self.head == None

    def travel_end(self):
        # 遍历到尾节点
        cur = self.head
        while cur.next != self.head:
            cur = cur.next
        return cur

    def append(self, item, name):
        # 尾插法
        node = Node(item, name)
        # 节点为空
        if self.is_empty():
            self.head = node
            node.next = node
        else:
            cur = self.travel_end()
            cur.next = node
            node.next = self.head

    def remove(self, item):
        # 删除结点
        if self.is_empty():
            return
        cur = self.head
        pre = None

        while cur.next != self.head:
            if cur.elem == item:
                # 删除位置在头节点且多于一个元素的情况
                if cur == self.head:
                    # 先找尾节点
                    popnode = cur
                    rear = self.travel_end()
                    rear.next = cur.next
                    self.head = cur.next
                else:
                    # 中间结点和尾结点的情况
                    popnode = cur
                    pre.next = cur.next
                # 执行完后直接返回减少计算量
                return popnode
            else:
                pre = cur
                cur = cur.next

        # 退出循环指向尾结点
        if cur.elem == item:
            #  一个元素的情况删掉后变为空节点  防止pre为none导致出错
            if cur == self.head:
                popnode = cur
                self.head = None
            else:
                # 尾节点元素的情况
                popnode = cur
                pre.next = self.head
        return popnode


class CycleOut(SingleCycleLinkList):
    def __init__(self, step, location):
        # 不初始化无法使用父类的head
        SingleCycleLinkList.__init__(self)
        self.step = step
        self.location = location

    def reset(self):
        cur = self.head
        if not self.is_empty():
            for each in range(self.location - 1):
                cur = cur.next
        return cur

    # 原来必须用循环输出到链表中，使用yield在while中可以从上次返回的地方继续执行可以一次只输出一个
    def outelem(self):
        count = 0
        cur = self.reset()
        # 不判断是否为空会出错
        while not self.is_empty():
            count += 1
            if count == self.step:
                popelem = self.remove(cur.elem)
                count = 0
                yield popelem
            cur = cur.next


def read_txt(path, mode):
    data = []
    with open(path, mode, encoding='UTF-8') as f:
        for line in f.readlines():
            line = line.rstrip()
            h = line.split()
            data.append(h)
        return data


def read_csv(path, mode):
    data = []
    # csv似乎不能用utf-8
    with open(path, mode, encoding='gbk') as f:
        for line in f.readlines():
            line = line.rstrip()
            h = line.split(',')
            data.append(h)
        return data


def read_zip(filename, keyword_file, mode):
    import zipfile
    z_file = zipfile.ZipFile(filename, mode)
    flist = z_file.namelist()
    if keyword_file not in flist:
        print('keyword file not in zip file')
    h = z_file.extract(keyword_file)
    data = []
    with open(h, mode, encoding='UTF-8') as f:
        for line in f.readlines():
            line = line.rstrip()
            h = line.split()
            data.append(h)
        return data


try:
    steps = int(input("数几个数"))
    if steps <= 0:
        raise Exception('数的数必须大于1')
    loc = int(input("开始位置"))
    if loc <= 0:
        raise Exception('位置必须大于1')
    joseph = CycleOut(steps, loc)
    ringdata = read_zip('peopledata.zip', 'peopledata.txt', 'r')
    for i in range(len(ringdata)):
        temp = ringdata[i]
        joseph.append(temp[1], temp[0])
except ValueError:
    print('输入错误，输入的不是数字')
else:
    popseq = joseph.outelem()
    # print(popseq.__next__().name) for循环一直执行next功能
    for n in popseq:
        print(n.name, int(n.elem))



输入错误，输入的不是数字
